In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from xgboost import XGBClassifier,XGBRegressor
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
import sklearn.metrics as metrics

In [4]:
from sklearn.metrics import roc_auc_score, roc_curve

In [22]:
from datetime import datetime

In [20]:
import pandas as pd 
import numpy as np

In [6]:
train_df=pd.read_pickle('feature/'+'2022_08_02_02_07_48'+'_train.pkl')

In [7]:
train_df

,id,loading,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,...,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,product_code,failure
0,0,80.10,7.0,8.0,4.0,18.040,12.518000,15.748,19.292,11.739,...,15.859,17.59400,15.193,15.029,13.638146,13.034,14.684000,764.100,A,0.0
1,1,84.89,14.0,3.0,3.0,18.213,11.540000,17.717,17.893,12.748,...,17.947,17.91500,11.755,14.732,15.425000,14.395,15.631000,682.057,A,0.0
2,2,82.43,12.0,1.0,5.0,18.057,11.652000,16.738,18.240,12.718,...,15.607,15.93469,13.798,16.711,18.631000,14.094,17.946000,663.376,A,0.0
3,3,101.07,13.0,2.0,6.0,17.295,11.188000,18.576,18.339,12.583,...,16.346,18.37700,10.020,15.250,15.562000,16.154,17.172000,826.282,A,0.0
4,4,188.06,9.0,2.0,8.0,19.346,12.950000,16.990,15.746,11.306,...,17.082,19.93200,12.428,16.182,12.760000,13.153,16.412000,579.885,A,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26565,26565,158.95,6.0,16.0,4.0,16.301,13.259000,18.068,15.505,10.865,...,12.177,17.94200,10.112,15.795,18.572000,16.144,13.279117,729.131,E,0.0
26566,26566,146.02,10.0,12.0,8.0,17.543,10.036905,17.984,19.078,11.139,...,14.179,20.56400,10.234,14.450,14.322000,13.146,16.471000,853.924,E,0.0
26567,26567,115.62,1.0,10.0,1.0,15.670,11.535000,16.778,18.385,11.630,...,16.437,17.47600,8.668,15.069,16.599000,15.590,14.065000,750.364,E,0.0
26568,26568,106.38,2.0,9.0,4.0,18.059,10.031023,16.918,18.101,11.713,...,17.064,17.81400,14.928,16.273,15.485000,13.624,12.865000,730.156,E,0.0


In [8]:
X=train_df.iloc[:,1:-2]
y=train_df.iloc[:,-1]

In [9]:
X.head()

,loading,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,80.10,7.0,8.0,4.0,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.59400,15.193,15.029,13.638146,13.034,14.684,764.100
1,84.89,14.0,3.0,3.0,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.91500,11.755,14.732,15.425000,14.395,15.631,682.057
2,82.43,12.0,1.0,5.0,18.057,11.652,16.738,18.240,12.718,18.288,12.715,15.607,15.93469,13.798,16.711,18.631000,14.094,17.946,663.376
3,101.07,13.0,2.0,6.0,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.37700,10.020,15.250,15.562000,16.154,17.172,826.282
4,188.06,9.0,2.0,8.0,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.93200,12.428,16.182,12.760000,13.153,16.412,579.885


In [ ]:
ssss

# 分類

In [13]:
# 建立 XGBClassifier 模型
xgboostModel = XGBClassifier(
        eval_metric="logloss",
        max_depth=11,
        learning_rate=0.0642,
        n_estimators=800,
        min_child_weight=7,
        grow_policy="depthwise",
        gamma=1.2840e-06)

In [14]:
avg_auc=0
k_fold_num=5
kf = GroupKFold(n_splits=k_fold_num)
for train_index, test_index in kf.split(X,y,train_df.product_code):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    xgboostModel.fit(X_train, y_train)
    predicted = xgboostModel.predict(X_train)
    accuracy = accuracy_score(y_train, predicted)
    print('train accuracy:',accuracy)
    
    predicted_test=xgboostModel.predict_proba(X_test)
    fpr, tpr, thresholds = metrics.roc_curve(y_test.values, predicted_test[:,1], pos_label=1)
    test_score=auc(fpr, tpr)
    print('test auc:',test_score)
    avg_auc=avg_auc+test_score
avg_auc=avg_auc/k_fold_num
print("================")
print('avg auc:',avg_auc)

train accuracy: 0.9990867579908675
test auc: 0.5278248480585763
train accuracy: 0.9994817920572855
test auc: 0.538294350851301
train accuracy: 0.999765478424015
test auc: 0.5377291286634772
train accuracy: 0.9997203840059652
test auc: 0.5419046762589929
train accuracy: 0.9993479273404751
test auc: 0.5399673221431122
avg auc: 0.5371440651950918


In [ ]:
ssss

# 回歸

In [17]:
xgboostModel = XGBRegressor(
        max_depth=10, 
        learning_rate=0.1, 
        n_estimators=800, 
        silent=True)

In [18]:
avg_auc=0
k_fold_num=5
kf = GroupKFold(n_splits=k_fold_num)
for train_index, test_index in kf.split(X,y,train_df.product_code):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    xgboostModel.fit(X_train, y_train)
    predicted = xgboostModel.predict(X_train)
    
    
    roc = roc_auc_score(y_train, predicted)
    print('train rmse:',roc)
    
    predicted_test=xgboostModel.predict(X_test)
    test_score = roc_auc_score(y_test, predicted_test)

    print('test auc:',test_score)
    avg_auc=avg_auc+test_score
avg_auc=avg_auc/k_fold_num
print("================")
print('avg auc:',avg_auc)

[15:56:49] WARNING: D:\Build\xgboost\xgboost-1.4.2.git\src\learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


train rmse: 1.0
test auc: 0.5246660895597756
[15:57:15] WARNING: D:\Build\xgboost\xgboost-1.4.2.git\src\learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


train rmse: 1.0
test auc: 0.5216174482866643
[15:57:43] WARNING: D:\Build\xgboost\xgboost-1.4.2.git\src\learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in la

In [ ]:
sss

## 邏輯回歸

In [12]:
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()
avg_auc=0
k_fold_num=5
kf = GroupKFold(n_splits=k_fold_num)
for train_index, test_index in kf.split(X,y,train_df.product_code):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    classifier.fit(X_train, y_train)
    predicted = classifier.predict(X_train)
    
    predicted_test=classifier.predict_proba(X_test)
    fpr, tpr, thresholds = metrics.roc_curve(y_test.values, predicted_test[:,1], pos_label=1)
    test_score=auc(fpr, tpr)
    print('test auc:',test_score)
    avg_auc=avg_auc+test_score
avg_auc=avg_auc/k_fold_num
print("================")
print('avg auc:',avg_auc)

test auc: 0.5820377283630002
test auc: 0.583260743104853
test auc: 0.5888953095535263
test auc: 0.5890438399280575
test auc: 0.5875892888614397
avg auc: 0.5861653819621753


# Save Model

In [23]:
now = datetime.now()
current_time = now.strftime("%Y_%m_%d_%H_%M_%S")
print("Current Time =", current_time)

Current Time = 2022_08_04_16_10_13


In [25]:
import pickle

In [26]:
file_name = current_time+"_logistic_model.pkl"
# save
pickle.dump(classifier, open("model/"+file_name, "wb"))